In [ ]:

import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split



# Load the FER-2013 dataset
df = pd.read_csv('fer2013.csv')

X = []
y = []



In [6]:
# Process each row in the dataset
for index, row in df.iterrows():
    pixels = np.asarray(row['pixels'].split(), dtype=np.uint8)  # Convert pixel values to numpy array
    pixels = pixels.reshape(48, 48, 1)  # Reshape to 48x48 grayscale image
    X.append(pixels)
    y.append(row['emotion'])



In [7]:
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)



In [8]:
# Normalize the pixel values (scale to [0, 1])
X = X / 255.0




In [9]:
# One-hot encode the labels (emotions)
y = to_categorical(y, num_classes=7)


In [10]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:

# Build the CNN model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral



C:\Users\RIA MALHOTRA\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [12]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])



In [13]:
# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 78s 170ms/step - accuracy: 0.2804 - loss: 1.7628 - val_accuracy: 0.4205 - val_loss: 1.5086
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 89s 198ms/step - accuracy: 0.4391 - loss: 1.4586 - val_accuracy: 0.4778 - val_loss: 1.3482
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 71s 158ms/step - accuracy: 0.5056 - loss: 1.2980 - val_accuracy: 0.5242 - val_loss: 1.2444
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 71s 159ms/step - accuracy: 0.5499 - loss: 1.1969 - val_accuracy: 0.5531 - val_loss: 1.1861
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 72s 161ms/step - accuracy: 0.5812 - loss: 1.1076 - val_accuracy: 0.5587 - val_loss: 1.1671
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 83s 184ms/step - accuracy: 0.6219 - loss: 1.0097 - val_accuracy: 0.5652 - val_loss: 1.1670
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 98s 218ms/step - accuracy: 0.6622 - loss: 0.9040 - val_accuracy: 0.5726 - val_loss: 1.1625
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 91s 202ms/step - accuracy: 0.7047 - loss: 0

In [2]:
pip install h5py

In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model



# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the face from the frame
        face = frame[y:y + h, x:x + w]

        # Convert to grayscale and resize to 48x48 (required by the model)
        face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face_resized = cv2.resize(face_gray, (48, 48))
        face_resized = face_resized.reshape(1, 48, 48, 1)
        face_resized = face_resized / 255.0

        # Predict the emotion
        emotion = model.predict(face_resized)
        max_index = np.argmax(emotion[0])
        emotion_text = emotion_labels[max_index]

        # Draw rectangle around the face and label the emotion
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Emotion Recognition', frame)

    # Press 'q' to quit the webcam feed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━